In [1]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.

This is by no means production code.
"""
# built-in imports
import re
from collections import defaultdict
from json import dump
from pprint import pprint
from urllib.request import urlopen
from xml.dom.minidom import Attr


import requests

# user packages
from bs4 import BeautifulSoup

# constants
BASE_URL = "https://www.domain.com.au"
N_PAGES = range(1, 51)  # update this to your liking
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36"
}

# begin code
url_links = []
property_metadata = defaultdict(dict)

# generate list of urls to visit
for page in N_PAGES:

    url = BASE_URL + f"/rent/melbourne-region-vic/?sort=price-desc&page={page}"
    bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")

    # find the unordered list (ul) elements which are the results, then
    # find all href (a) tags that are from the base_url website.
    index_links = bs_object.find("ul", {"data-testid": "results"}).findAll(
        "a", href=re.compile(f"{BASE_URL}/*")  # the `*` denotes wildcard any
    )

    for link in index_links:
        # if its a property address, add it to the list
        if "address" in link["class"]:
            url_links.append(link["href"])


# for each url, scrape some basic metadata
for property_url in url_links[1:]:
    bs_object = BeautifulSoup(
        requests.get(property_url, headers=headers).text, "html.parser"
    )

    # looks for the header class to get property name
    property_metadata[property_url]["name"] = bs_object.find(
        "h1", {"class": "css-164r41r"}
    ).text

    # looks for the div containing a summary title for cost
    property_metadata[property_url]["cost_text"] = bs_object.find(
        "div", {"data-testid": "listing-details__summary-title"}
    ).text

    
    # extract coordinates from the hyperlink provided
    # i'll let you figure out what this does :P
    property_metadata[property_url]["coordinates"] = [
        float(coord)
        for coord in re.findall(
            r"destination=([-\s,\d\.]+)",  # use regex101.com here if you need to
            bs_object.find(
                "a", {"target": "_blank", "rel": "noopener noreferer"}
            ).attrs["href"],
        )[0].split(",")
    ]
    

    attr_dict = [feature.text for feature in bs_object \
            .find("div", {"data-testid": "property-features"}) \
            .findAll("span", {"data-testid": "property-features-text-container"})]
    
    if attr_dict == []:
        property_metadata[property_url]['rooms'] = 'null'
        property_metadata[property_url]['baths'] = 'null'
        property_metadata[property_url]['parking'] = 'null'
        property_metadata[property_url]['area'] = 'null'
    else:
        property_metadata[property_url]['rooms'] = re.findall(r'\d\s[A-Za-z]+', attr_dict[0])
        property_metadata[property_url]['baths'] = re.findall(r'\d\s[A-Za-z]+', attr_dict[1])
        property_metadata[property_url]['parking'] = re.findall(r'\d\s[A-Za-z]+', attr_dict[2])
        if len(attr_dict) == 4:
            property_metadata[property_url]['area'] = attr_dict[3]


    property_metadata[property_url]["desc"] = re.sub(
        r"<br\/>", "\n", str(bs_object.find("p"))
    ).strip("</p>")




# output to example json in data/raw/
with open("../data/raw/example.json", "w") as f:
    dump(property_metadata, f)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [2]:
import pandas as pd 

df=pd.read_json("../data/raw/example.json")

df




,https://www.domain.com.au/667-glenhuntly-road-caulfield-vic-3162-11598047,https://www.domain.com.au/upstairs-2c-staley-street-brunswick-vic-3056-13168913,https://www.domain.com.au/level-3-302-13-15-lake-street-caroline-springs-vic-3023-15994395,https://www.domain.com.au/9-lansdowne-street-blairgowrie-vic-3942-12127675,https://www.domain.com.au/52-black-st-brighton-vic-3186-15410646,https://www.domain.com.au/3502-14-16-the-esplanade-st-kilda-vic-3182-16002767,https://www.domain.com.au/4203-35-spring-street-melbourne-vic-3000-15939303,https://www.domain.com.au/440-keys-road-flinders-vic-3929-16065322,https://www.domain.com.au/335-the-esplanade-indented-head-vic-3223-12688424,https://www.domain.com.au/50-south-wharf-drive-docklands-vic-3008-16048359,...,https://www.domain.com.au/2-91-brighton-rd-elwood-vic-3184-9223720,https://www.domain.com.au/9-gertrude-street-geelong-west-vic-3218-15996931,https://www.domain.com.au/1-bluff-rd-merricks-beach-vic-3926-15995340,https://www.domain.com.au/1-11-bay-street-parkdale-vic-3195-16050233,https://www.domain.com.au/2-261-williams-road-south-yarra-vic-3141-16043406,https://www.domain.com.au/level-1-301-13-15-lake-street-caroline-springs-vic-3023-15994278,https://www.domain.com.au/1-235-ascot-vale-road-ascot-vale-vic-3032-16052405,https://www.domain.com.au/45-murdock-street-brunswick-vic-3056-16035344,https://www.domain.com.au/17-owen-street-brunswick-west-vic-3055-15882530,https://www.domain.com.au/46-alder-st-caulfield-south-vic-3162-16073205
name,667 Glenhuntly Road Caulfield VIC 3162,Upstairs 2C Staley Street Brunswick VIC 3056,"Level 3, 302/13-15 Lake Street Caroline Spring...",9 Lansdowne Street Blairgowrie VIC 3942,52 Black St Brighton VIC 3186,3502/14-16 The Esplanade St Kilda VIC 3182,4203/35 Spring Street Melbourne VIC 3000,440 Keys Road Flinders VIC 3929,335 The Esplanade Indented Head VIC 3223,50 South Wharf Drive Docklands VIC 3008,...,2/91 Brighton Rd Elwood VIC 3184,9 Gertrude Street Geelong West VIC 3218,1 Bluff Rd Merricks Beach VIC 3926,1/11 Bay Street Parkdale VIC 3195,2/261 Williams Road South Yarra VIC 3141,"Level 1, 301/13-15 Lake Street Caroline Spring...",1/235 Ascot Vale Road Ascot Vale VIC 3032,45 Murdock Street Brunswick VIC 3056,17 Owen Street Brunswick West VIC 3055,46 Alder St Caulfield South VIC 3162
cost_text,"$38,000 p.a. Incl. Outgoings + GST","$35,000 Annually",4125000 pw,"$5,000 per week",From $5500 per week,"$5,000",$4500 Per Week,"$4,000.00","$3,775 wk $545 pn",$3750 per week,...,$850 per week,$795,$795,$795,$795 per week,$41250 pa,$790.00,$790.00,$790.00,$790 weekly
coordinates,"[-37.8860233, 145.0173065]","[-37.7655919, 144.9633048]","[-37.7316459, 144.7446886]","[-38.372703, 144.7856897]","[-37.9159452, 144.9989003]","[-37.8650177, 144.9746821]","[-37.8141725, 144.9740049]","[-38.4788173, 144.9643428]","[-38.1439923, 144.7147899]","[-37.8223967, 144.938237]",...,"[-37.8733009, 144.9893772]","[-38.1449175, 144.3465017]","[-38.3997481, 145.1048468]","[-38.00308, 145.0810804]","[-37.8420314, 145.004217]","[-37.7316459, 144.7446886]","[-37.77483489999999, 144.9233434]","[-37.757776, 144.9587099]","[-37.766182, 144.938694]","[-37.890215, 145.018058]"
rooms,null,[0 Beds],null,[3 Beds],[3 Beds],[3 Beds],[3 Beds],[6 Beds],[4 Beds],[5 Beds],...,[3 Beds],[4 Beds],[3 Beds],[3 Beds],[2 Beds],null,[3 Beds],[4 Beds],[4 Beds],[3 Beds]
baths,null,[0 Baths],null,[2 Baths],[4 Baths],[3 Baths],[2 Baths],[4 Baths],[3 Baths],[6 Baths],...,[2 Baths],[2 Baths],[2 Baths],[2 Baths],[1 Bath],null,[3 Baths],[1 Bath],[2 Baths],[2 Baths]
parking,null,[2 Parking],null,[3 Parking],[3 Parking],[3 Parking],[4 Parking],[2 Parking],[],[3 Parking],...,[2 Parking],[3 Parking],[2 Parking],[2 Parking],[1 Parking],null,[2 Parking],[2 Parking],[1 Parking],[2 Parking]
area,null,NaN,null,700m²,900m²,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,null,NaN,NaN,NaN,NaN
desc,Can you hear it? Opportunity knocking!!,1st floor offices/studios\nEasy walking distan...,- Centrally located with tranquil vie